In [3]:
import sys
sys.path.insert(0, '../')

import torch
import torchvision
import torchdrift
import copy
%matplotlib inline
from matplotlib import pyplot

device =  "cpu"

from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds import get_dataset

import numpy as np

In [ ]:
resnet = torchvision.models.resnet18(pretrained=True)
model = torch.nn.Sequential(
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    resnet,
    resnet.fc
    )
resnet.fc = torch.nn.Identity()
model.eval().to(device)
for p in model.parameters():
    p.requires_grad_(False)

In [ ]:
# datensatz camelyon laden
%cd  '/home/jinglewsl/evoila/projects/image-drift-monitoring'
%pwd
camelyon = get_dataset(dataset="camelyon17",download=False)

WILDS_PATH = 'data'
N = 5000 # Size of reference set
DOWNLOAD = False
img_size = (96,96)
ds_train = camelyon.get_subset('train',transform=transforms.Compose([transforms.Resize(img_size), transforms.ToTensor()]))
train_loader = iter(get_train_loader('standard',ds_train,1))

ds_test = camelyon.get_subset('test',transform=transforms.Compose([transforms.Resize(img_size), transforms.ToTensor()]))
test_loader = iter(get_train_loader('standard',ds_test,1))

x_ref = np.stack([next(train_loader)[0][0].numpy() for _ in range(N)],axis=0)
x_test = np.stack([next(test_loader)[0][0].numpy() for _ in range(N)],axis=0)

In [ ]:
val_transform =  torchvision.transforms.ToTensor()


ds_train = torchvision.datasets.ImageFolder('/home/jinglewsl/evoila/projects/image-drift-monitoring/image_data/ants_vs_bees/train/',
                                            transform=val_transform)
ds_val = torchvision.datasets.ImageFolder('/home/jinglewsl/evoila/projects/image-drift-monitoring/image_data/ants_vs_bees/val/',
                                          transform=val_transform)
dl_val = torch.utils.data.DataLoader(ds_val, batch_size=64, shuffle=True)

In [ ]:
def fit_detector(N_train):
    detector = torchdrift.detectors.KernelMMDDriftDetector(return_p_value=True)
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=N_train, shuffle=True)
    feature_extractor = model[:2]  # without the fc layer
    torchdrift.utils.fit(dl_train, feature_extractor, detector, num_batches=1)
    return detector

detector = fit_detector(N_train = 100)